In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import plotly
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py 
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from builder import *

import keras
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers.core import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2, l1

from sklearn.model_selection import train_test_split

/Users/bychkov/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:

 This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.


Using TensorFlow backend.


In [3]:
data_x, data_y = load_surv_samples('simulated_data/linear_5K.pkl', sort=False)

train_x, test_x, train_y, test_y = train_test_split(
        data_x, data_y, test_size=0.2, random_state=75 )

train_x, val_x, train_y, val_y = train_test_split(
        train_x, train_y, test_size=0.25, random_state=75 )

print("Train split: %d; Val split: %d; Test split: %d." % (train_x.shape[0],val_x.shape[0],test_x.shape[0]))

Loading data from: simulated_data/linear_5K.pkl
Train split: 3000; Val split: 1000; Test split: 1000.


In [28]:
model = Sequential()
model.add( InputLayer(input_shape=(5, ), name='InputLayer') )

model.add( Dense(2, kernel_regularizer=l1(.01)) )
model.add( Activation('sigmoid') )
model.add( Dense(3, kernel_regularizer=l1(.01)) )
model.add( Activation('sigmoid') )
model.add( Dense(1, activation='linear') )

model.compile(loss      = partial_likelihood,
              optimizer = 'adadelta')


In [29]:
opts                    = dict()
opts['dump_freq']       = 10
opts['plot_freq']       = 10
opts['krs_model']       = model
opts['loss']            = partial_likelihood
opts['metrics_l']       = None
opts['optimizer']       = 'adadelta'
opts['exp_path']        = '/Users/bychkov/GDD/projects/simu/models'
opts['exp_name']        = 'mlp_1'
opts['continue']        = False

In [30]:
network = Builder(  opts = opts,
                    train_data = ( train_x, train_y ),
                    test_data  = ( test_x, test_y ),
                    val_data   = ( val_x, val_y )  ) 

#history = network.train(epoches=15, batch=25, verbose=1)


initializing a new model...
Model initialized.


In [42]:
history = network.train(epoches=100, batch=train_x.shape[0], shuffle=True, verbose=0)

In [313]:
dfs   = list() 

# Train:
d  = OrderedDict()
d['id']    = train_y[:,3]
d['h_hat'] = np.squeeze( network.model.predict(train_x, batch_size=3000, verbose=1) )
dfs.append( pd.DataFrame( data=d ) )

# Validation:     
d  = OrderedDict()
d['id']    = val_y[:,3]
d['h_hat'] = np.squeeze( network.model.predict(val_x, batch_size=1000, verbose=1) )
dfs.append( pd.DataFrame( data=d ) )

# Test:     
d  = OrderedDict()
d['id']    = test_y[:,3]
d['h_hat'] = np.squeeze( network.model.predict(test_x, batch_size=1000, verbose=1) )
dfs.append( pd.DataFrame( data=d ) )
            
df = pd.concat(dfs, ignore_index=True, axis=0)
df.sort_values(by='id', inplace=True)
df.to_csv('models/mlp_1/preds.csv', index = False)

1000/1000 [==============================] - 0s
